In [6]:
# Update and install required packages for this demonstration.
!{__import__('sys').executable} -m pip install -U setuptools pip 'marrow.mongo'

In [7]:
class classinstancemethod:
	"""A decorator to allow protocol methods to behave as class and instance methods."""
	
	def __init__(self, fn):
		self.fn = staticmethod(fn)
	
    def __get__(self, obj, cls=None):
        # There's probably a way, way, way better way of doing this!
        return partial(self.fn, cls if obj is None else obj)
	
	def __set__(self, obj, value):
		raise TypeError("Dual class/instance methods can not be written to.")


class Depend:
	NS = 'org.python.setuptools.entry_point'
	PLUGIN = 'org.marrow.package.load'
	ASSET = 'web.component.asset'
	BUNDLE = 'web.asset.bundle'
	STYLE = 'web.asset.selector'
	EVENT = 'web.asset.event'
	
	ALL = (NS, PLUGIN, ASSET, BUNDLE, STYLE, EVENT)
	
	@classmethod
	def collect(cls, obj, *only):
		if not only:
			only = cls.ALL
		
		yield from obj.__depend__(obj, only)
	
	@classmethod
	def declare(cls, fn):
		return classinstancemethod(fn)

In [8]:
from marrow.mongo import Document, utcnow
from marrow.mongo.field import Date, Link


class Address(Document):
	"""An embedded document representing a verified e-mail address.
	
	Intended for use in an Array() embedding these sub-documents.
	"""
	
	__pk__ = 'address'
	__vary__ = ('address', 'verified')  # The attributes to fetch for cache key generation.
	
	address = Link(required=True, protocols={'mailto'})  # The e-mail address itself.
	added = Date(default=utcnow, write=False)  # When was this address added?
	verified = Date(default=None, write=False)  # Has the user verified the address, and if so, when?
	
	@Depend.declare  # self may actually be the class, not an instance!
	def __depends__(self, only=Depend.ALL):
		"""Declare bundle, themeing, and interaction points for this widget."""
		
		if Depend.STYLE in only:
			# Hard theme selectors; this is always present.
			yield Depend.STYLE, '.email'
			
			# Conditional theme selectors; announce all anyway, but hint.
			if issubclass(self, Address):  # Class-level identification.
				yield Depend.STYLE, '.verified'
				yield Depend.STYLE, '.unverified'
			else:  # Conditional actual use when querying an instance specifically.
				yield Depend.STYLE, '.verified', bool(self.verified)
				yield Depend.STYLE, '.unverified', not self.verified
		
		if Depend.EVENT in only:
			# Client- and server-side events that may fire.
			yield Depend.EVENT, 'com.illico.common.email.verified'
	
	def __str__(self):
		"""Return a plain text representation of this location; just the name, if present."""
		
		return self.address.path.name
	
	def __html__(self):
		"""Return a microdata-compliant e-mail link.
		
		If the e-mail address is fully-formed with a name component, the name will be used for presentation.
		"""
		
		return f'<a href="mailto:{self.address}" class="email {"" if self.verified else 'un'}verified">{self}</a>'
	
	def __link__(self):
		"""Return a valid URI representitive of this widget."""
		
		return self.address

SyntaxError: f-string: expecting '}' (2792648936.py, line 49)